## Homework 7

**Ruixue Gong, N17593858**

**Exercise 1**

$$q(x,y): = \mathbb{P}\{X_{t+1} = y |X_t = x\}$$
and 
$$p(x) = \beta \sum\limits_{y\in S}[d(y)+p(y)]q(x,y)$$

By Theorem 0.1, 
$$d_{\infty}(T^kp, p^*) \rightarrow 0$$

**Banach Fixed Point Theorem**
Let $(X, d)$ be a non-empty complete metric space with a contraction mapping $T : X \rightarrow X$. Then T admits a unique fixed-point $x^*$ in X (i.e. $T(x^*) = x^*$). Furthermore, $x^*$ can be found as follows: start with an arbitrary element $x_0$ in X and define a sequence $\{x_n\}$ by $x_n = T(x_{n−1})$, then $x_n \rightarrow x^*$.


(1)Provide a Python or Julia function that takes a specification of
the primitives and returns an approximation to the equilibrium price function.

From Theorem 0.1, we have already dnow that T is a uniform contraction mapping. Therefore we can use Banach Fixed Point Theorem to compute fixed point function $p(y)$.

Experiment with different tolerance and choose a number that gives you accuracy up to three or four decimal places.

In [19]:
type Lucas_Asset_Price
    β::Float64                #discount rate
    tol::Float64              #tolerance level
    maxit::Int64              #maximum iteration number
    price::Array{Float64,2}   #prices of different states
    t_mat::Array{Float64,2}   #transitional probability matrix
    d_val::Function           #dividend function
end

Define a consant vector function for divident values.
Notice that this function is given


$\displaystyle  d_{val }(y)= (1,0,0)'$

In [20]:
d_val(lap::Lucas_Asset_Price) = [1.0 0.0 0.0]'

d_val (generic function with 1 method)

Define a uniform contraction mapping as 
$$Tp(y): = \beta \sum\limits_{y\in S}[d(y)+p(y)]q(x,y) $$

In [21]:
function T_map!(lap::Lucas_Asset_Price, val::Array{Float64,2})
    lap.price = lap.β * lap.t_mat * (val+lap.price) 
    # 2\times 2 * 2\times 1    
end 

T_map! (generic function with 1 method)

Now we need to show the iteration process 

$$p_n = T(p_{n-1})$$ 

In [22]:
function iter!(lap::Lucas_Asset_Price)
 
    dval = lap.d_val(lap)
    iter_count = 0
    err = 2.0
    
    while err > lap.tol
        p_prior = lap.price
        iter_count += 1
        if iter_count > lap.maxit
            break
        else
            T_map!(lap, dval)
            p_post = lap.price
            err = norm(p_post - p_prior)
        end
    end
end


iter! (generic function with 1 method)

In [23]:
function main(t_mat::Array{Float64,2})  #input initial price
    p_0 = zeros(3,1)
    
    #set parameters
    β = 0.98   #as suggested by hw7
    tol = 1e-10
    maxit = 100000
   
    lap = Lucas_Asset_Price(β,tol,maxit,p_0,t_mat,d_val) #transition 
    iter!(lap)    #iteration
    
    return lap.price
end

main (generic function with 1 method)

In [24]:
t_mat1 = [[.971,.145,.0] [.029,.778,.508] [.0,.077,.492]]
t_mat2 = [[.871,.145,.0] [.029,.778,.508] [.1,.077,.492]]

p1 = round(main(t_mat1)',3)
p2 = round(main(t_mat2)',3)

println("The equilibirum prices of Case 1 is ", p1)
println("The equilibirum prices of Case 2 is ", p2)


The equilibirum prices of Case 1 is [40.76 35.961 34.572]
The equilibirum prices of Case 2 is [23.501 21.099 20.284]


**Analysis**:

There is a huge slump of prices from case 1 to case 2. And we notice that in case 2, that there exists a possitive probability of 0.1 transitioning from normal growth to severe recession. However in case 1, if we are in normal growth, it's impossible to jump to severe recession. Thus the unconditional probability of severe recession would be increased in case 2, which results in the slump of asset prices. Since dividend function is fixed for each state, the price for each state will be only influenced by transition probability. The higher the probability of normal growth, the higher the price will be.

**Exercise 2**

The equilibrium condition is modified to 
$$p(x) = \beta \max_{i \in \{1,2\}} \{\sum\limits_{y\in S}[d(y)+p(y)]q_i(x,y)\}$$
where $q_1$ and $q_2$ represent different beliefs about state transition probabilities

Proof: 

Note that I is an arbitrary index set. And S is not finite now. 

Define a map T such that 
$$\displaystyle Tp(x) := \beta \sup_{i \in I} \{\sum\limits_{y\in S}[d(y)+p(y)]q_i(x,y)\}, \quad\forall p\in b\mathbb{R}^S$$

We want to show that T is a uniform contraction. 
We have shown that 
$$|Tp(x)| = \beta\sup_{i \in I} \{ |\sum\limits_{y\in S}[d(y)+p(y)]q(x,y)|\}$$
$$\leq \beta \sup_{i \in I}\{ \sum\limits_{y\in S}|d(y)+p(y)|q(x,y)\}$$
$$\leq \beta \sup_{i \in I}\{ \sum\limits_{y\in S}(|d(y)|+|p(y)|)q(x,y)\}$$
$$\leq \beta \sup_{i \in I}\{ \sum\limits_{y\in S}(\parallel d(y)\parallel_{\infty}+\parallel p(y)\parallel_{\infty})q(x,y)\}$$
$$\leq  \beta (\parallel d(y)\parallel_{\infty}+\parallel p(y)\parallel_{\infty})$$

Thus $\parallel Tp \parallel_{\infty} < \infty$. $Tp\in b\mathbb{R}^S$

The next step we need to show that 
$$\parallel Tp-Tp'\parallel_{\infty} \leq \beta \parallel p-p'\parallel_{\infty}$$

Consider 
$$|Tp(x)-Tp'(x)| = \beta|\sup_{i \in I} \{ \sum\limits_{y\in S}[d(y)+p(y)]q(x,y)\}-\sup_{i \in I} \{ \sum\limits_{y\in S}[d(y)+p'(y)]q(x,y)\}|$$

Claim: 
$$|\sup_i a_i -\sup_i b_i| \leq \sup_i |a_i-b_i|$$

Given any $\epsilon>0$, there exists $m\in I $ such that 
$|a_m- b_m|> \sup_i|a_i-b_i|-\epsilon \geq|\sup_i a_i -\sup_i b_i|-\epsilon $
And notice that $\sup_i|a_i-b_i|\geq |a_m -b_m |>|\sup_i a_i -\sup_i b_i|-\epsilon $

Since $\epsilon$ is arbitrary, we can conclude that 
$$\sup_i|a_i-b_i|\geq |\sup_i a_i -\sup_i b_i|$$

Therefore by the claim we just proved, we can show
$$\beta|\sup_{i \in I} \{ \sum\limits_{y\in S}[d(y)+p(y)]q(x,y)\}-\sup_{i \in I} \{ \sum\limits_{y\in S}[d(y)+p'(y)]q(x,y)\}|$$
$$\leq \beta \sup_i |\sum\limits_{y\in S}p(y)-p'(y)q(x,y)|$$
$$\leq \beta \sup_i|p(y)-p'(y)| $$

Thus T is a uniform contraction on $(b\mathbb{R}^S,d_\infty)$

By Banach contraction mappint theorem, there exists a unique fixed point $p^*$.

**Exercise 3**

Following the setting written on hwset7.pdf, there are two agents. $i= 1,2$

In [25]:
type Lucas_Asset_Price_hetero
    β::Float64                #discount rate
    tol::Float64              #tolerance level
    maxit::Int64              #maximum iteration number
    price::Array{Float64,2}   #prices of different states
    t_mat_1::Array{Float64,2} #transitional probability matrix 1
    t_mat_2::Array{Float64,2} #transitional probability matrix 2
    d_val::Function           #dividend function
end

In [26]:
d_val(lap::Lucas_Asset_Price_hetero) = [1.0 0.0 0.0]'

d_val (generic function with 2 methods)

In [27]:
function T_map!(lap::Lucas_Asset_Price_hetero, val::Array{Float64,2})
    lap.price = lap.β*max(lap.t_mat_1*(val+lap.price),lap.t_mat_2*(val+lap.price))
    # 2\times 2 * 2\times 1    
end 

T_map! (generic function with 2 methods)

In [28]:
function iter!(lap::Lucas_Asset_Price_hetero)
 
    dval = lap.d_val(lap)
    iter_count = 0
    err = 2.0
    
    while err > lap.tol
        p_prior = lap.price
        iter_count += 1
        if iter_count > lap.maxit
            break
        else
            T_map!(lap, dval)
            p_post = lap.price
            err = norm(p_post - p_prior)
        end
    end
end

iter! (generic function with 2 methods)

In [34]:
function main2(t_mat_1::Array{Float64,2},t_mat_2::Array{Float64,2})  #input initial price
    p_0 = zeros(3,1)
    
    #set parameters
    β = 0.98   #as suggested by hw7
    tol = 1e-10
    maxit = 100000
   
    lap = Lucas_Asset_Price_hetero(β,tol,maxit,p_0,t_mat_1,t_mat_2,d_val) #transition 
    iter!(lap)    #iteration
    
    return lap.price
end

main2 (generic function with 1 method)

In [41]:
t_mat_1 = [[.971,.145,.0] [.029,.778,.508] [.0,.077,.492]]
t_mat_2 = [[.871,.145,.0] [.029,.778,.508] [.1,.077,.492]]

p = round(main2(t_mat_1, t_mat_2)')

println("The equilibirum prices with heterogenious agenets is ", p)
println("Recall case 1 and case 2 in homogenious agent model")
println(p1)
println(p2)


The equilibirum prices with heterogenious agenets is [41.0 36.0 35.0]
Recall case 1 and case 2 in homogenious agent model
[40.76 35.961 34.572]
[23.501 21.099 20.284]


**Analysis**:

Since heterogenious model pick the maximum value of asset prices of these two agents, so it is higher than the value of case 1 and that of case 2